In [1]:
import sys
sys.path.append("..")

In [2]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"
database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [3]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint='https://sales-chatbot-llm.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-3-small',
    openai_api_version='2023-05-15'
)

In [4]:
# from langchain_community.vectorstores import Neo4jVector
# # ! pip3 install -U langchain-huggingface
# import os
# os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
# from langchain_huggingface import HuggingFaceEmbeddings
# # Choose from https://huggingface.co/spaces/mteb/leaderboard

# # embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

# model_path = os.path.join(os.environ['SENTENCE_TRANSFORMERS_HOME'], 'models--lier007--xiaobu-embedding-v2/snapshots/ee0b4ecdf5eb449e8240f2e3de2e10eeae877691')
# embedding = HuggingFaceEmbeddings(model_name=model_path)

In [5]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)<-[:HAS_CHILD]->(p:__Parent__)
    WITH c, p, count(distinct n) as freq
    RETURN p.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WHERE c.summary is not null
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    match (n)
    WHERE n.description is not null
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['id', 'description'],
                                    retrieval_query=lc_retrieval_query)
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

local_search_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9,
                   'k': topEntities,
                   'params': {
                        "topChunks": topChunks,
                        "topCommunities": topCommunities,
                        "topOutsideRels": topOutsideRels,
                        "topInsideRels": topInsideRels,
                    }},
    tags=['GraphRAG']
)
local_search_retriever

VectorStoreRetriever(tags=['GraphRAG'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x15739efc0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 10, 'params': {'topChunks': 3, 'topCommunities': 3, 'topOutsideRels': 10, 'topInsideRels': 10}})

In [6]:
# print(local_search_retriever.invoke('保險費暨保險單借款利息自動轉帳付款授權')[0].page_content)

In [7]:
from tools.TWLF_Neo4jVector import TWLF_Neo4jVector
vectorstore = TWLF_Neo4jVector.from_existing_graph(
                                    embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='__Chunk__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
vector_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9},
    tags=['RAG']
)

In [8]:
import os
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1574703b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x157471ac0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x147f0bbf0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x157470410>, model_kwargs={}, openai_api_key=SecretStr('**********'), azure_endpoint='https://sales-chatbot-llm.openai.azure.com/', deployment_name='GPT4o', openai_api_version='2023-03-15-preview', openai_api_type='azure')

In [9]:
# from langchain_ollama import ChatOllama
# llm = ChatOllama(
#     # model="llama3.1:70b-instruct-q8_0",
#     model='qwen2:72b-instruct-q8_0',
# )
# llm

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
"""
你是一個有用的助手, 你的任務是整理提供的資訊, 使用長度與格式符合「multiple paragraphs」針對使用者的問題來回應,
提供的資訊包含 檢索內容、圖譜資料庫相關節點與關係資訊, 無關的資訊直接忽略
你必須使用繁體中文回應問題, 盡可能在500字內回應,
若提供的資訊全部無關聯, 回應「找不到相關資訊」並結束, 不要捏造任何資訊,
最終的回應將清理後的訊息合併成一個全面的答案，針對回應的長度和格式對所有關鍵點和含義進行解釋
根據回應的長度和格式適當添加段落和評論。以Markdown格式撰寫回應。
回應應保留原有的意思和使用的情態動詞，例如「應該」、「可以」或「將」。
請確保使用繁體中文回答問題


以下為檢索內容:
"{context}"

以下為圖譜資料庫相關節點(Entities)、關係(Relationships)、社群(Reports)、Chunks(內文節錄)資訊:
"{graph_result}"

問題: {question}
"""
)

rag_chain = (
    {"context": vector_retriever, "question": RunnablePassthrough(), "graph_result": local_search_retriever}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
import sys
sys.path.append('..')
from tools.TokenCounter import num_tokens_from_string

q = '首期匯款帳號有哪些銀行, 匯款帳號跟劃撥帳號是什麼?'
print(num_tokens_from_string(q))

32


In [17]:
for r in rag_chain.stream(q):
    print(r, end="", flush=True)

No relevant docs were retrieved using the relevance score threshold 0.9


根據提供的資訊，首期保險費的匯款帳號可以分為台幣和外幣兩種，以下為詳細資訊：

### 台幣匯款帳號
- **匯款銀行**：中國信託商業銀行城中分行
- **匯入帳號**：3562+1碼(類別)+10碼(保單號碼)+1碼(檢核碼)，共計16碼

### 外幣匯款帳號
- **匯款銀行**：CTBC Bank Co., Ltd (中國信託銀行)
- **戶名**：台灣人壽保險股份有限公司
- **SWIFT CODE**：CTCBTWTPXXX
- **虛擬帳號**：
  - **類型一**：95432+保單號碼10碼+檢查碼，共16碼
  - **類型二**：95265+保單號碼10碼，共15碼

這些帳號分別用於不同幣別的保單繳費，保戶需要根據保單類型選擇適合的匯款帳號進行繳費。請注意，匯款時需要在劃撥單或匯款單上註明保單號碼、要保人姓名、聯絡電話及相關款項清償金額。

此外，保戶須自行負擔匯款手續費，並確保繳款人是保單關係人，如非保單關係人，需提供相關身份或關係證明文件。

### 郵局劃撥
- **劃撥帳號**：50120507 (至郵局劃撥至本公司之郵政劃撥帳號)

### 匯款銀行詳細列表
- **中國信託銀行**
- **兆豐銀行**
- **第一銀行**
- **彰化銀行**
- **台新銀行**
- **元大銀行**
- **永豐銀行**
- **匯豐銀行（美金）**
- **華南銀行**
- **國泰世華**
- **合作金庫**

每個銀行都有各自的識別碼、SWIFT Code及詳細的跨行匯款規定，保戶可以依據自身需求選擇合適的銀行進行匯款。

### 注意事項
1. 請在劃撥單或匯款單上註明保單號碼、要保人姓名、聯絡電話及借款/墊繳本息清償金額。
2. 繳費完成後，還款收據將由本公司郵寄給保戶。
3. 若有需要，保戶可聯繫客服專線或訪問本公司會員網站查詢虛擬帳號轉換碼。

希望這些資訊能夠幫助您順利完成首期保險費的匯款。

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = (
    "Given a chat history and the latest user question " # 給定一段聊天歷史和使用者的最新問題
    "which might reference context in the chat history, " # 這個問題可能會引用聊天歷史中的上下文
    "formulate a standalone question which can be understood " # 請將問題重新表述為一個獨立的問題，使其在沒有聊天歷史的情況下也能被理解
    "without the chat history. Do NOT answer the question, " # 不要回答這個問題
    "just reformulate it if needed and otherwise return it as is." # 只需在必要時重新表述問題，否則原樣返回
    "請確保使用繁體中文回應"
)

# 定義上下文解析的Chain
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{question}"),
    ]
)

contextualize_chain = (
    contextualize_q_prompt
    | llm
    | StrOutputParser().with_config({
        'tags': ['contextualize_question']
    })
)


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableParallel, RunnableLambda

store = {}

    
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 使用 RunnableParallel 來組織多個並行查詢
context_and_search_chain = RunnableParallel(
    {
        "context": RunnableLambda(lambda inputs: vector_retriever.invoke(inputs)),
        "graph_result": RunnableLambda(lambda inputs: local_search_retriever.invoke(inputs)),
        "question": lambda x: x,  # 保留原始輸入
    }
)

rag_chain = (
    contextualize_chain
    | context_and_search_chain
    | prompt
    | llm
    | StrOutputParser().with_config({
        "tags": ['final_output']
    })
)
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)


In [ ]:

# for r in conversational_rag_chain.stream(
#     {"input": "常見的繳費方式為何有這三種?"},
#     config={
#         "configurable": {"session_id": "abc123"}
#     },  # constructs a key "abc123" in `store`.
# ):
#     print(r , end="")

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser
from fastapi import FastAPI
from langserve import add_routes
from typing import List, Tuple
from pydantic import BaseModel, Field

class ChatHistory(BaseModel):
    """Chat history with the bot."""
    question: str
    
conversational_rag_chain = (
  conversational_rag_chain | StrOutputParser()
).with_types(input_type=ChatHistory)

# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    conversational_rag_chain
)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)